<img>
<html>
  <head>
  </head>

  <body>
    <img src="images/spam.jpg">
  </body>
</html>

I've got an email classification dataset, where each row is an email and every column contains a word that has been found in the emails with values indicating how many words there are. This summary of the data sets is provided:

Number of Rows: 5172 (each row represents an email)
Number of Columns: 3002 (1 column for email name, 3000 columns for words, 1 column for labels)
First Column: Email name (encoded with numbers for privacy)
Last Column: Labels for prediction (1 for spam, 0 for not spam)
Remaining Columns: 3000 most common words in the emails (after excluding non-alphabetical characters/words)
This data set is appropriate for text classification and, in particular, to detect spam. I've used techniques like Naive Bayes and Decision Trees to create a model predicting whether an email is spam or not because of the word count in it.

---
👋 Greetings, all! *I'm Sajjad Ali Shah*, and I'd want to introduce myself to you. I'm a young data scientist that appreciates using data to acquire valuable insights and make critical decisions. With a strong interest in exploring the domains of data science, I actively participate in many projects and offer my knowledge on platforms such as Kaggle.
🔍 As a young Data Scientist, I'm always seeking for new ways to analyze, understand, and visualize data to solve real-world challenges. My passion for unraveling patterns, discovering trends, and uncovering hidden insights in complex datasets drives my profession in data science.


- Contact: [Linkedin](www.linkedin.com/in/sajjad-ali-shah-120341305)

- Date : 18/09/2024
- Dataset: [Link](https://www.kaggle.com/datasets/venky73/spam-mails-dataset)
---

# Import the libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load the dataset

In [5]:
# load data through pandas 
df=pd.read_csv("./spam dataset/spam_ham_dataset.csv")

In [6]:
# load first 5 dataset
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
